# Make Nowcast Timeseries
This notebook describes the process of extracting a timeseries of SalishSeaCast Nowcast and HRDPS results for analysis.

In [1]:
from salishsea_tools import grid_tools, timeseries_tools
import xarray as xr
import pickle

## Functions
`reshape_GEM` flattens the HRDPS records downloaded from ERDDAP to 2-D (time and space)

In [24]:
def reshape_GEM(data, grid, mask):
    """Flattens GEM HRDPS wind to 2-D (time-space) and removes land points
    """

    # Initialize storage
    data_flat = {'raw': {'grid': {}}}
    
    # Build GEM coordinates
    mask_flt, coords, ngrid, ngrid_water = timeseries_tools.reshape_coords_GEM(
        grid, mask,
    )

    # Reshape, flatten, and trim
    data_flat['raw']['grid']['u'] = timeseries_tools.reshape_to_ts(
        data.u_wind.values, mask_flt.astype(bool), ngrid, ngrid_water,
    )
    data_flat['raw']['grid']['v'] = timeseries_tools.reshape_to_ts(
        data.v_wind.values, mask_flt.astype(bool), ngrid, ngrid_water,
    )

    # Assign parameters to output dict
    data_flat['mask'] = mask_flt
    data_flat['coords'] = coords
    data_flat['ngrid'] = ngrid
    data_flat['ngrid_water'] = ngrid_water

    return data_flat

`iterate_NEMO_timeseries` is a wrapper for `timeseries_tools.load_NEMO_timeseries` that loads and flattens Nowcast results to 2-D (time and space).

**Note** For optimization purposes, `timeseries_tools.load_NEMO_timeseries` slices along a single dimension by definition. If 3-D (x, y, depth) or 1-D are desired, consider `xarray.open_mfdataset` or ERRDAP. If averages along a dimension are desired, consider modifying `timeseries_tools.load_NEMO_timeseries`.

In [26]:
def iterate_NEMO_timeseries(timerange, variables, mask, dims, indices):
    """Wrapper for timeseries_tools.load_NEMO_timeseries that loads and flattens
    Nowcast results to 2-D (time-space) and removes land points
    """
    
    # Intialize empty dict
    results = {}

    # Loop through results slices
    for var in variables:  # --- Loop through variables
        for dim, index in zip(dims, indices):  # --- Loop through slices
            
            # Populate dict fields
            results[var] = {}
            results[var][dim] = {}
            results[var][dim][index] = {}

            # Determine filenames and unstaggering from variable names
            dim_in, unstagger = dim, None
            if var is 'vozocrtx':
                file = 'U'
                if dim is 'depth':
                    dim_in, unstagger = 'depthu', 'x'
            elif var is 'vomecrty':
                file = 'V'
                if dim is 'depth':
                    dim_in, unstagger = 'depthv', 'y'
            else:
                file = 'T'
                if dim is 'depth':
                    dim_in = 'deptht'

            # Determine spacing from variable names and slicing
            if (var is 'vozocrtx' or var is 'vomecrty') and dim is 'depth':
                spacing = 5
            else:
                spacing = 1

            # Construct Filename List
            filenames = timeseries_tools.make_filename_list(
                timerange, file, model='nowcast-green', resolution='h'
            )
            
            # Load Results
            data, coords = timeseries_tools.load_NEMO_timeseries(
                filenames, mask, var, dim_in, index=index,
                spacing=spacing, shape='flat', unstagger_dim=unstagger,
            )
            
            # Export results as dict
            results[var][dim][index]['data'] = data
            results[var][dim][index]['coords'] = coords

            # Export timerange
            results[var][dim][index]['coords']['timerange'] = timerange

    return results

## Load Results
First specify a timerange

In [4]:
# Timerange
timerange = ['2017 Jan 1 00:00', '2017 Jan 31 23:00']

Next load the grid and mask files

In [5]:
# Load mask and grid
grid_NEMO = xr.open_dataset('/data/bmoorema/MEOPAR/NEMO-forcing/grid/bathy_downonegrid2.nc')
grid_GEM = xr.open_dataset('/results/forcing/atmospheric/GEM2.5/operational/ops_y2017m01d01.nc')
mask_NEMO = xr.open_dataset('/data/bmoorema/MEOPAR/NEMO-forcing/grid/mesh_mask_downbyone2.nc')

# Build GEM mask
mask_GEM = grid_tools.build_GEM_mask(grid_GEM, grid_NEMO, mask_NEMO.tmask.isel(t=0, z=0))

Building GEM mask 100% (68096 of 68096) |###################################|Time: 0:03:59


Next load the GEM record from ERDDAP and flatten with `reshape_GEM`

In [7]:
# Load GEM record
GEM_in = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaSurfaceAtmosphereFieldsV1')
GEM = reshape_GEM(GEM_in.sel(time=slice(*timerange)), grid_GEM, mask_GEM)

Next load flatten the Nowcast record for each requested variable and slice. Save to a `pickle` file.

In [27]:
# Loop through requests
dims = ['depth', 'depth', 'y', 'y', 'y']
indices = [0, 20, 450, 520, 680]
variables = ['votemper', 'vosaline', 'vozocrtx', 'vomecrty']
Nowcast = iterate_NEMO_timeseries(timerange, variables, mask_NEMO, dims, indices)

# Save output
with open('/ocean/bmoorema/research/MEOPAR/analysis-ben/data/NowcastJan2017', 'wb') as fid:
    pickle.dump(Nowcast, fid)

Loading votemper, deptht=0 100% (31 of 31) |################################|Time: 0:00:42
Loading votemper, deptht=20 100% (31 of 31) |###############################|Time: 0:00:40
Loading votemper, y=450 100% (31 of 31) |###################################|Time: 0:00:39
Loading votemper, y=520 100% (31 of 31) |###################################|Time: 0:00:39
Loading votemper, y=680 100% (31 of 31) |###################################|Time: 0:00:37
Loading vosaline, deptht=0 100% (31 of 31) |################################|Time: 0:00:43
Loading vosaline, deptht=20 100% (31 of 31) |###############################|Time: 0:00:40
Loading vosaline, y=450 100% (31 of 31) |###################################|Time: 0:00:41
Loading vosaline, y=520 100% (31 of 31) |###################################|Time: 0:00:40
Loading vosaline, y=680 100% (31 of 31) |###################################|Time: 0:00:36
Loading vozocrtx, depthu=0 100% (31 of 31) |################################|Time: 0:01:18

TypeError: write() argument must be str, not bytes